# University Rankings Database Operations

This notebook demonstrates CRUD (Create, Read, Update, Delete) operations on a SQLite database containing university rankings data from 2012 to 2015.

## 1. Database Connection Setup

First, let's import the required libraries and establish a connection to our SQLite database.

In [31]:
import sqlite3
import pandas as pd
from typing import List, Tuple, Any

def connect_to_db(db_path: str) -> sqlite3.Connection:
    """Establish a connection to the SQLite database."""
    try:
        conn = sqlite3.connect(db_path)
        print("Successfully connected to the database!")
        return conn
    except sqlite3.Error as e:
        print(f"Error connecting to database: {e}")
        raise

# Connect to the database
db_path = "university_database.db"
conn = connect_to_db(db_path)

Successfully connected to the database!


## 2. Data Exploration and Basic Analysis

Let's explore the database structure and perform some basic analysis on the university rankings data.

In [23]:

def explore_database(conn: sqlite3.Connection) -> None:
    """Explore the database structure and show basic statistics."""
    cursor = conn.cursor()
    
    # Get table information
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables in the database:")
    for table in tables:
        print(f"- {table[0]}")
        
    # Sample the data
    cursor.execute(f"SELECT * FROM university_rankings LIMIT 5;")
    columns = [description[0] for description in cursor.description]
    rows = cursor.fetchall()
    
    print("\nTable structure:")
    print("Columns:", columns)
    print("\nSample data:")
    for row in rows:
        print(row)

# Explore the database
explore_database(conn)

# Basic statistics
query = """
SELECT 
    year,
    COUNT(*) as total_universities,
    AVG(score) as avg_score,
    MIN(score) as min_score,
    MAX(score) as max_score
FROM university_rankings
GROUP BY year
ORDER BY year;
"""

df = pd.read_sql_query(query, conn)
print("\nBasic statistics by year:")
print(df)

Tables in the database:
- university_rankings

Table structure:
Columns: ['world_rank', 'institution', 'country', 'national_rank', 'quality_of_education', 'alumni_employment', 'quality_of_faculty', 'publications', 'influence', 'citations', 'broad_impact', 'patents', 'score', 'year']

Sample data:
(1, 'Harvard University', 'USA', 1, 7, 9, 1, 1, 1, 1, '', 5, 100.0, 2012)
(2, 'Massachusetts Institute of Technology', 'USA', 2, 9, 17, 3, 12, 4, 4, '', 1, 91.67, 2012)
(3, 'Stanford University', 'USA', 3, 17, 11, 5, 4, 2, 2, '', 15, 89.5, 2012)
(4, 'University of Cambridge', 'United Kingdom', 1, 10, 24, 4, 16, 16, 11, '', 50, 86.17, 2012)
(5, 'California Institute of Technology', 'USA', 4, 2, 29, 7, 37, 22, 22, '', 18, 85.21, 2012)

Basic statistics by year:
   year  total_universities  avg_score  min_score  max_score
0  2012                 100   54.94090      43.36      100.0
1  2013                 100   55.27120      44.26      100.0
2  2014                1000   47.27141      44.18      

## 3. Insert Operations

Let's add Duke Tech to the rankings for 2014.

In [25]:
def insert_university(conn: sqlite3.Connection, 
                     institution: str, 
                     country: str, 
                     year: int, 
                     world_rank: int, 
                     score: float) -> None:
    """Insert a new university into the rankings database."""
    cursor = conn.cursor()
    try:
        cursor.execute("""
            INSERT INTO university_rankings (institution, country, year, world_rank, score)
            VALUES (?, ?, ?, ?, ?);
        """, (institution, country, year, world_rank, score))
        conn.commit()
        print(f"Successfully inserted {institution} into the university_rankings for {year}")
    except sqlite3.Error as e:
        print(f"Error inserting data: {e}")
        conn.rollback()

# Insert Duke Tech's 2014 ranking
insert_university(
    conn,
    institution="Duke Tech",
    country="USA",
    year=2014,
    world_rank=350,
    score=60.5
)

# Verify the insertion
cursor = conn.cursor()
cursor.execute("""
    SELECT * FROM university_rankings 
    WHERE institution = 'Duke Tech' 
    AND year = 2014;
""")
result = cursor.fetchone()
print("\nVerification - Inserted Data:")
print(result)

Successfully inserted Duke Tech into the university_rankings for 2014

Verification - Inserted Data:
(350, 'Duke Tech', 'USA', None, None, None, None, None, None, None, None, None, 60.5, 2014)


## 4. Query Operations

Let's find out how many Japanese universities are in the top 200 for 2013.

In [26]:
def count_top_universities(conn: sqlite3.Connection, 
                          country: str, 
                          year: int, 
                          rank_limit: int) -> int:
    """Count universities from a specific country in top N for a given year."""
    cursor = conn.cursor()
    cursor.execute("""
        SELECT COUNT(*) 
        FROM university_rankings
        WHERE country = ? 
        AND year = ?
        AND world_rank <= ?;
    """, (country, year, rank_limit))
    return cursor.fetchone()[0]

# Count Japanese universities in top 200 for 2013
japan_count = count_top_universities(conn, "Japan", 2013, 200)
print(f"Number of Japanese universities in the global top 200 in 2013: {japan_count}")

# Show the details of these universities
cursor = conn.cursor()
cursor.execute("""
    SELECT institution, world_rank, score
    FROM university_rankings
    WHERE country = 'Japan'
    AND year = 2013
    AND world_rank <= 200
    ORDER BY world_rank;
""")
print("\nDetails of Japanese universities in top 200 (2013):")
for uni in cursor.fetchall():
    print(f"Institution: {uni[0]}, Rank: {uni[1]}, Score: {uni[2]}")

Number of Japanese universities in the global top 200 in 2013: 6

Details of Japanese universities in top 200 (2013):
Institution: University of Tokyo, Rank: 14, Score: 76.23
Institution: Kyoto University, Rank: 15, Score: 69.46
Institution: Osaka University, Rank: 35, Score: 52.79
Institution: Keio University, Rank: 70, Score: 47.05
Institution: Nagoya University, Rank: 91, Score: 44.78
Institution: Tohoku University, Rank: 95, Score: 44.52


## 5. Update Operations

Let's update the University of Oxford's score for 2014 by adding 1.2 points.

In [27]:
def update_university_score(conn: sqlite3.Connection,
                          institution: str,
                          year: int,
                          score_change: float) -> None:
    """Update a university's score by adding the specified amount."""
    cursor = conn.cursor()
    
    # First get the current score
    cursor.execute("""
        SELECT score 
        FROM university_rankings 
        WHERE institution = ? 
        AND year = ?;
    """, (institution, year))
    current_score = cursor.fetchone()
    
    if current_score is None:
        print(f"No record found for {institution} in {year}")
        return
    
    # Update the score
    try:
        cursor.execute("""
            UPDATE university_rankings 
            SET score = score + ? 
            WHERE institution = ? 
            AND year = ?;
        """, (score_change, institution, year))
        conn.commit()
        print(f"Successfully updated {institution}'s score for {year}")
        
        # Show the change
        cursor.execute("""
            SELECT score 
            FROM university_rankings 
            WHERE institution = ? 
            AND year = ?;
        """, (institution, year))
        new_score = cursor.fetchone()[0]
        print(f"Old score: {current_score[0]}, New score: {new_score}")
    except sqlite3.Error as e:
        print(f"Error updating score: {e}")
        conn.rollback()

# Update University of Oxford's score
update_university_score(conn, "University of Oxford", 2014, 1.2)

Successfully updated University of Oxford's score for 2014
Old score: 97.51, New score: 98.71000000000001


## 6. Delete Operations

Let's remove universities with scores below 45 in 2015.

In [29]:
def delete_low_scoring_universities(conn: sqlite3.Connection,
                               year: int,
                               score_threshold: float) -> None:
    """Delete universities with scores below the threshold for a given year."""
    cursor = conn.cursor()
    
    # First, let's see how many universities will be affected
    cursor.execute("""
        SELECT COUNT(*) 
        FROM university_rankings 
        WHERE year = ? 
        AND score < ?;
    """, (year, score_threshold))
    count = cursor.fetchone()[0]
    
    # Show the universities that will be deleted
    cursor.execute("""
        SELECT institution, score 
        FROM university_rankings 
        WHERE year = ? 
        AND score < ?;
    """, (year, score_threshold))
    to_be_deleted = cursor.fetchall()
    
    print(f"Found {count} universities with scores below {score_threshold} in {year}")
    print("\nUniversities to be removed:")
    for uni in to_be_deleted:
        print(f"- {uni[0]} (Score: {uni[1]})")
    
    # Perform the deletion
    try:
        cursor.execute("""
            DELETE FROM university_rankings 
            WHERE year = ? 
            AND score < ?;
        """, (year, score_threshold))
        conn.commit()
        print(f"\nSuccessfully removed {count} universities")
    except sqlite3.Error as e:
        print(f"Error deleting records: {e}")
        conn.rollback()

# Delete universities with scores below 45 in 2015
delete_low_scoring_universities(conn, 2015, 45.0)

# Verify the deletion
cursor = conn.cursor()
cursor.execute("""
    SELECT MIN(score) 
    FROM university_rankings 
    WHERE year = 2015;
""")
min_score = cursor.fetchone()[0]
print(f"\nVerification - Minimum score in 2015 after deletion: {min_score}")

Found 556 universities with scores below 45.0 in 2015

Universities to be removed:
- University of Dayton (Score: 44.99)
- University of Siena (Score: 44.98)
- University of Oulu (Score: 44.96)
- Indian Institute of Science (Score: 44.96)
- Macquarie University (Score: 44.96)
- University of Rostock (Score: 44.96)
- Jagiellonian University (Score: 44.94)
- Pusan National University (Score: 44.94)
- University of Strathclyde (Score: 44.94)
- University of Nevada, Reno (Score: 44.93)
- University of Saskatchewan (Score: 44.92)
- University of Parma (Score: 44.92)
- Kanazawa University (Score: 44.92)
- Eötvös Loránd University (Score: 44.92)
- Aristotle University of Thessaloniki (Score: 44.92)
- Ohio University (Score: 44.91)
- University of Iceland (Score: 44.91)
- University of Warsaw (Score: 44.91)
- Syracuse University (Score: 44.91)
- Bielefeld University (Score: 44.91)
- Juntendo University (Score: 44.9)
- University of Wyoming (Score: 44.89)
- University of Wollongong (Score: 44.8

## 7. Cleanup

Close the database connection properly.

In [30]:
# Close the database connection
conn.close()
print("Database connection closed.")

Database connection closed.
